# Introduction

A decision tree is a non-parametric supervised learning algorithm, which is utilized for both classification and regression tasks. It has a hierarchical, tree structure, which consists of a root node, branches, internal nodes and leaf nodes.

The model starts with a root node, which does not have any incoming branches. The outgoing branches from the root node then feed into the internal nodes, also known as decision nodes. Based on the available features, both node types conduct evaluations to form homogenous subsets, which are denoted by leaf nodes, or terminal nodes. The leaf nodes represent all the possible outcomes within the dataset.
Decision tree learning employs a divide and conquer strategy by conducting a greedy search to identify the optimal split points within a tree. This process of splitting is then repeated in a top-down, recursive manner until all, or the majority of records have been classified under specific class labels.

Two methods are populary used to choose the best attribute at each node, Entropy and Gini impurity.

Entropy measures the impurity of the sample values. Its  values can fall between 0 and 1. If all samples in data set, S, belong to one class, then entropy will equal zero. If half of the samples are classified as one class and the other half are in another class, entropy will be at its highest at 1.

In order to select the best feature to split on and find the optimal decision tree, the attribute with the smallest amount of entropy should be used. Information gain represents the difference in entropy before and after a split on a given attribute. The attribute with the highest information gain will produce the best split as it is doing the best job at classifying the training data according to its target classification.

Secondly, Gini impurity is the probability of incorrectly classifying random data point in the dataset if it were labeled based on the class distribution of the dataset.


To make these concepts work and create a decision tree algorithm from scratch, this code is arranged as follows:

1-  Determines if the tree is for regresion or classification.

2 - Identifies potential split values for each feature, considering both categorical and continuous data types.

3- Computes evaluation metrics like Gini impurity or entropy for classification trees and variance for regression trees to assess the quality of potential splits.

4- Splits the node based on a specific feature and its value, creating left and right child nodes. Recursively splits nodes until a stopping criterion (depth or score threshold) is met, forming the decision tree.


###References:


https://www.ibm.com/topics/decision-trees#:~:text=A%20decision%20tree%20is%20a,internal%20nodes%20and%20leaf%20nodes.

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

# Data

Attribute Information:

1) ID number
2) Diagnosis (M = malignant, B = benign)
3-32)

Ten real-valued features are computed for each cell nucleus:

3) radius (mean of distances from center to points on the perimeter)

4) texture (standard deviation of gray-scale values)

5) perimeter

6) area

7) smoothness (local variation in radius lengths)

8) compactness (perimeter^2 / area - 1.0)

9) concavity (severity of concave portions of the contour)

10) concave points (number of concave portions of the contour)

11) symmetry

12) fractal dimension ("coastline approximation" - 1)


The mean, standard error and "worst" or largest (mean of the three
largest values) of these features were computed for each image,
resulting in 30 features. For instance, field 3 is Mean Radius, field
13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant







###Reference:
http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from more_itertools import set_partitions
from sklearn.tree import DecisionTreeClassifier

import more_itertools

In [11]:

class DecisionTreeNode:

    def __init__(self,features_mat,y_labels,depth,metrics_type='Gini',score_to_stop_split=0.9):
        self.right_node = None
        self.left_node = None
        self.feature_index_split = None
        self.value_split = None
        self.depth_in_tree = depth
        self.features_mat = features_mat
        self.y_labels = y_labels
        self.label_of_leaf = None
        self.metrics_type = metrics_type
        self.score_to_stop_split = score_to_stop_split
        self.is_classification_tree = self._check_if_is_classification_tree()
        self.is_trained = False

    def _check_if_is_classification_tree(self):
       #This fucntion evaluates if it a clasification or regression
        return True if self.y_labels.dtype==object else False

    def _split_one_node(self,feature_index,value_to_split):
        if type(value_to_split)==list:  #for categorical features
            first_group = value_to_split[0] #takes the first value
            mask = np.isin(self.features_mat[:,feature_index],first_group) #use np.isin to identify data points where the feature value belongs to the first category.
        else:
            mask = self.features_mat[:,feature_index]<=value_to_split #Array true and false. continues values
        right_features_mat = self.features_mat[mask]
        left_features_mat = self.features_mat[~mask]
        right_labels = self.y_labels[mask]
        left_labels = self.y_labels[~mask]
        return left_features_mat,right_features_mat,right_labels,left_labels

    def _get_features_to_check(self,num_features):
      #return a sequence of numbers of columns
        return range(num_features)

    def _get_all_split_values(self,feature_index):
      # Identifies and prepares all potential split points for a given feature in the tree algorithm, accounting for both categorical and continuous data types.

        values = self.features_mat[:,feature_index] #takes all the values from one feature of the dataset
        unique_values = np.unique(values) #Extract the unique values

        if type(values[0])==str: #If the feature is categorical
            split_values = list(set_partitions(unique_values,2)) #it uses set_partitions to generate all possible combinations of splitting the unique values into two groups.

        else: #If the feature is continuous (numeric type)
            sorted_values = np.sort(unique_values)
            if len(unique_values)<10000:
                #Creates split points by taking the midpoint between adjacent values
                sorted_values_remove_first = sorted_values[1:] #take values from second element to end
                sorted_values_no_last = sorted_values[:-1] #from beguining witho out the last one.
                split_values = (sorted_values_no_last+sorted_values_remove_first)/2
            else:
              #if the unique value exceeds 10,000, it divides the continuous range of values into percentile ranges from 1 to 99 percentiles, with a step of 1
               split_values = np.percentile(sorted_values,range(1,99,1))
        return split_values


    def _calcualte_metrics(self,right_labels,left_labels):
      #this function evaluates the quality of the potential splits using evaluation metrics such as "Gini" or "Entropy" for classification trees, and "Variance" for regression trees

        is_gini = self.metrics_type == 'Gini' #defines metric
        metrics = 0
        total_size = len(left_labels) + len(right_labels) #get lenght

        for branch_labels in [right_labels,left_labels]:

            if self.is_classification_tree: #if it is a clasification tree
                vals,counts = np.unique(branch_labels,return_counts=True) #get unique values and their corresponding counts
                probabilities = counts/counts.sum() #calculates the probability of occurrence for each unique value
                gini_of_branch = 1-(probabilities**2).sum() #it calculates the Gini impurity. A lower Gini impurity suggests a better separation of classes in that particular node of the tree
                entropy_of_branch = -(probabilities*np.log2(probabilities)).sum() #calculate the entropy of a branch
                metrics_branch = gini_of_branch if is_gini else entropy_of_branch
            else:
                metrics_branch = np.var(branch_labels) #for regression tree, it calculates the variance of the branch labels.

            weight_of_branch = len(branch_labels)/total_size #Computes the weight of the current branch
            metrics += metrics_branch*weight_of_branch #it sums the weighted metrics for both branches. it represents the overall quality of the potential split
        return metrics

    def _check_all_options_for_split(self):
      #This function iterates through features and potential split values to find the best possible split point. Returns the datasets resulting from the best split


        num_columns = self.features_mat.shape[1] #number of features
        best_score = np.inf # positive infinity as initial best score
        features_to_search = self._get_features_to_check(num_columns) #get range of features


        for feature_index in features_to_search: #loop through features to search
            split_values = self._get_all_split_values(feature_index) #get potential split

            for value in split_values: #Loop through the potential split values
                left_features_mat,right_features_mat,right_labels,left_labels = \
                                                self._split_one_node(feature_index,value)
                score = self._calcualte_metrics(right_labels,left_labels) # Calculate metrics for the split

                if score < best_score: # Check if the current split's score is better
                    best_score = score
                    best_feature = feature_index
                    best_value = value
                    best_right_features_mat = right_features_mat
                    best_left_features_mat = left_features_mat
                    best_left_labels = left_labels
                    best_right_labels = right_labels
        self.feature_index_split = best_feature
        self.value_split = best_value
        return best_right_features_mat,best_left_features_mat,best_left_labels,best_right_labels

    def _calc_score_and_label_of_node(self):
      #compute the score and prediction label for a node in a decision tree. It returns the calculated score and prediction label

        if self.is_classification_tree: #for clasification
            vals,freq = np.unique(self.y_labels,return_counts=True)   #calculates unique values and their frequencies

            #calculating purity
            probabilities = freq/freq.sum()  #probabilities of each unique label
            max_p = np.max(probabilities) #Finds the maximum probability among all classes.
            score = max_p

            #calculating label
            max_index = np.argmax(freq) #Obtains the index of the most frequent class.
            prediction = vals[max_index] #Assigns the label associated with the most frequent class as the prediction label.

        else: #For a Regression Tree:

            minus_var = -np.var(self.y_labels) #Calculates the negative variance of the target labels. Higher magnitude means more spread
            score = minus_var
            prediction = np.median(self.y_labels) #Computes the median of the target labels
        return score,prediction

    def _split_node_recursively(self,max_depth=6):
        # This function builds the decision tree recursively by splitting nodes until a stopping condition is met, at which point the node becomes a leaf by storing the final label value.

        self.is_trained = True #Marks the current node as trained.
        score,label_of_node = self._calc_score_and_label_of_node()

        #stopping criteria for tree
        if self.depth_in_tree > max_depth or score > self.score_to_stop_split: #Checks if the current node's depth exceeds the max_depth allowed
            self.label_of_leaf = label_of_node  #this is a leaf
            del self.features_mat,self.y_labels #Deletes the features and labels associated with this node

        else:
            right_features_mat,left_features_mat,left_labels,right_labels = \
                                                self._check_all_options_for_split()
            del self.features_mat,self.y_labels

            self.right_node = DecisionTreeNode(right_features_mat,right_labels,self.depth_in_tree+1,\
                    self.metrics_type,self.score_to_stop_split) #Initializes the right child node
            self.right_node._split_node_recursively(max_depth)


            self.left_node = DecisionTreeNode(left_features_mat,left_labels,self.depth_in_tree+1,\
                    self.metrics_type,self.score_to_stop_split) #Initializes the left child node
            self.left_node._split_node_recursively(max_depth)

    def fit(self,max_depth = 6):
        self._split_node_recursively(max_depth)


    def predict_numerical_features_only(self,row):
       if self.label != None:
           return self.label
       else:
           if row[self.feature_index] > self.value:
               return self.left.predict(row)
           else:
               return self.right.predict(row)

    def predict(self,data_row):
      #function to make predictions
       if not self.is_trained:
          print("Model not trained yet")
          return

       #stoppint criteria - if we are in a leaf
       if self.label_of_leaf != None:
           return self.label_of_leaf
       else:

           # categorical or numerical features
           if type(self.value_split)==list: #if the categorical alue is in the list

              if data_row[self.feature_index_split].isin(self.value_split[0]):
                    return self.left_node.predict(data_row)
              else:
                   return self.right_node.predict(data_row)

           else: #for numerical features
               if data_row[self.feature_index_split] > self.value_split:
                   return self.left_node.predict(data_row)
               else:
                   return self.right_node.predict(data_row)

    def print_tree(self):
        if self.label_of_leaf!=None:
            print('\t'*(self.depth_in_tree-1),'label is: ',self.label_of_leaf,\
                  'and depth is: ',self.depth_in_tree)
        else:
            print('\t'*(self.depth_in_tree-1),'split is at feature: ',\
                      self.feature_index_split,' and value ',self.value_split,\
                      ' and depth is: ',self.depth_in_tree)
        # if self.right_node:
            self.right_node.print_tree()
        # if self.left_node:
            self.left_node.print_tree()

    def save_tree(self,file_name = 'my_tree'):
        if not self.is_trained:
            print("Tree isn't trained yet")
        else:
            with open(file_name,'wb') as fid:
                pickle.dump(self,fid)


    def load_tree(file_name = 'my_tree'):
        with open(file_name,'rb') as fid:
            tree = pickle.load(fid)
        return tree

    def evalute(self,test_x,test_y):
      #gets accuracy
        test_labels = np.array([self.predict(row_data) for row_data in test_x])
        if self.is_classification_tree:
            metrics = 100*(test_labels==test_y).sum()/len(y_test)
        else:
            metrics = np.sqrt(np.mean( (test_labels-test_y)**2 ))
        return metrics


In [25]:
if __name__ == '__main__':
    #reading data and pre-processing
    file_name = r'/content/drive/MyDrive/Colab Notebooks/wdbc.data'
    data = pd.read_csv(file_name,header = None)

    y = data.iloc[:,1].values #target
    x = data.drop(1,axis=1) # features
    x = x.drop(0,axis=1).values #remove id from data
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8) #split

    root_node = DecisionTreeNode(x_train,y_train,1)
    root_node.fit(5)
    root_node.print_tree()
    accuracy = root_node.evalute(x_test,y_test)
    root_node.save_tree()
    print('\naccuracy is {}'.format(accuracy))

    new_tree = DecisionTreeNode.load_tree()
    prediction_loaded = new_tree.predict(x_test[0,:])

    sklern_tree = DecisionTreeClassifier(max_depth=5)
    sklern_tree.fit(x_train,y_train)
    sklearn_score = sklern_tree.score(x_test,y_test)
    print('\nsklearn accuracy is {}'.format(sklearn_score))



 split is at feature:  22  and value  105.95  and depth is:  1
	 label is:  B and depth is:  2
	 split is at feature:  21  and value  20.645  and depth is:  2
		 split is at feature:  7  and value  0.08365500000000001  and depth is:  3
			 label is:  B and depth is:  4
			 label is:  M and depth is:  4
		 label is:  M and depth is:  3

accuracy is 90.35087719298245

sklearn accuracy is 0.9385964912280702
